<h2>导入包</h2>

In [58]:
import logging
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from lightgbm import Dataset


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

<h2>读取数据</h2>

In [59]:
x_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_气象变量数据.csv", encoding="gbk")
y_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_实际功率数据.csv", encoding="gbk")
info_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_基本信息.csv", encoding="gbk")
x_test = pd.read_csv("../data/A榜-测试集_分布式光伏发电预测_气象变量数据.csv", encoding="gbk")
y_test = pd.read_csv("../data/submit_example.csv", encoding="utf-8")
info_test = pd.read_csv("../data/A榜-测试集_分布式光伏发电预测_基本信息.csv", encoding="gbk")

In [60]:
x_train = pd.merge(x_train, info_train[["光伏用户编号", "装机容量(kW)", "经度", "纬度"]], how="left", on="光伏用户编号")
x_train["时间"] = pd.to_datetime(x_train["时间"])
x_test = pd.merge(x_test, info_test[["光伏用户编号", "装机容量(kW)", "经度", "纬度"]], how="left", on="光伏用户编号")
x_test["时间"] = pd.to_datetime(x_test["时间"])

In [61]:
y_train = y_train.set_index(["光伏用户编号", "综合倍率", "时间"]).stack().reset_index().rename(columns={0:"target"})
y_train["level_3"] = y_train["level_3"].apply(lambda x: int(x[1:]))
y_train["时间"] = pd.to_datetime(y_train["时间"])
y_train["时间"] = y_train["时间"] + (y_train["level_3"] - 1) * 15 * pd.Timedelta(1, unit="minutes")
y_train = y_train.drop(columns=["level_3"])

y_test = y_test.set_index(["光伏用户编号", "综合倍率", "时间"]).stack().reset_index().rename(columns={0:"target"})
y_test["level_3"] = y_test["level_3"].apply(lambda x: int(x[1:]))
y_test["时间"] = pd.to_datetime(y_test["时间"])
y_test["时间"] = y_test["时间"] + (y_test["level_3"] - 1) * 15 * pd.Timedelta(1, unit="minutes")
y_test = y_test.drop(columns=["level_3"])

In [62]:
df_train = pd.merge(x_train, y_train, on=["光伏用户编号", "时间"], how="left")
df_test = pd.merge(x_test, y_test, on=["光伏用户编号", "时间"], how="left")

<h2>特征工程</h2>

<h3>时间特征</h3>

In [63]:
df_train["年"] = df_train["时间"].dt.year
df_train["月"] = df_train["时间"].dt.month
df_train["日"] = df_train["时间"].dt.day
df_train["分"] = df_train["时间"].dt.minute // 15 + df_train["时间"].dt.hour * 4
df_train = df_train.drop(columns=["时间"])

df_test["年"] = df_test["时间"].dt.year
df_test["月"] = df_test["时间"].dt.month
df_test["日"] = df_test["时间"].dt.day
df_test["分"] = df_test["时间"].dt.minute // 15 + df_test["时间"].dt.hour * 4
df_test = df_test.drop(columns=["时间"])

<h2>训练模型</h2>

<h3>评测指标</h3>

In [64]:
def score(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return 1 / (1 + rmse)

<h3>lightgbm模型</h3>

In [65]:
params_lgb = {
    'learning_rate': 0.02,
    'boosting_type': 'gbdt',
    'objective': 'mse',
    'metric': 'mse',
    'num_leaves': 64,
    'verbose': -1,
    'seed': 42,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
}

In [66]:
model_lgb = dict()
kfold = KFold(n_splits=5, random_state=42, shuffle=True)
for site, df in df_train.groupby("光伏用户编号"):
    model_lgb[site] = []
    df = df.drop(columns=["光伏用户编号"]).astype(np.float32)
    y = df.pop("target")
    df["辐照强度（J/m2） - 1"] = df["辐照强度（J/m2）"].shift(1)
    for fold, (train_index, val_index) in enumerate(kfold.split(df, y)):
        logging.info(f'############ site: {site} fold: {fold} ###########')
        x_train, x_val, y_train, y_val = df.iloc[train_index], df.iloc[val_index], y.iloc[train_index], y.iloc[val_index]
        trainset = Dataset(x_train, y_train)
        valset = Dataset(x_val, y_val)
        model = lgb.train(params_lgb, trainset, valid_sets=[trainset, valset], num_boost_round=5000, callbacks=[lgb.early_stopping(100), lgb.log_evaluation(1000)])
        model.save_model("../models/lgb_%s_%d.txt" % (site, fold))
        model_lgb[site].append(model)

2024-03-07 12:18:21,236 : INFO : ############ site: f1 fold: 0 ###########


Training until validation scores don't improve for 100 rounds
[1000]	training's l2: 0.00820665	valid_1's l2: 0.0192935
[2000]	training's l2: 0.00422071	valid_1's l2: 0.0184285
[3000]	training's l2: 0.00253238	valid_1's l2: 0.0182106
Early stopping, best iteration is:
[3338]	training's l2: 0.00217332	valid_1's l2: 0.0181561


2024-03-07 12:18:42,064 : INFO : ############ site: f1 fold: 1 ###########


Training until validation scores don't improve for 100 rounds
[1000]	training's l2: 0.00787395	valid_1's l2: 0.0223177
[2000]	training's l2: 0.00404532	valid_1's l2: 0.0210572
[3000]	training's l2: 0.00242543	valid_1's l2: 0.0206658
[4000]	training's l2: 0.00156003	valid_1's l2: 0.0205088
Early stopping, best iteration is:
[4301]	training's l2: 0.00138204	valid_1's l2: 0.0204609


2024-03-07 12:19:08,597 : INFO : ############ site: f1 fold: 2 ###########


...

Training until validation scores don't improve for 100 rounds
[1000]	training's l2: 0.000496794	valid_1's l2: 0.00113465
[2000]	training's l2: 0.000266079	valid_1's l2: 0.00106204
[3000]	training's l2: 0.000164187	valid_1's l2: 0.00104031
Early stopping, best iteration is:
[3840]	training's l2: 0.000115519	valid_1's l2: 0.00103199


2024-03-07 12:33:30,817 : INFO : ############ site: f9 fold: 3 ###########


Training until validation scores don't improve for 100 rounds
[1000]	training's l2: 0.000499326	valid_1's l2: 0.00113853
[2000]	training's l2: 0.000261786	valid_1's l2: 0.00105482
[3000]	training's l2: 0.000160151	valid_1's l2: 0.00103599
[4000]	training's l2: 0.000105165	valid_1's l2: 0.00102658
Early stopping, best iteration is:
[4436]	training's l2: 8.8715e-05	valid_1's l2: 0.00102508


2024-03-07 12:34:07,903 : INFO : ############ site: f9 fold: 4 ###########


Training until validation scores don't improve for 100 rounds
[1000]	training's l2: 0.000494027	valid_1's l2: 0.00116099
[2000]	training's l2: 0.000261729	valid_1's l2: 0.00107825
[3000]	training's l2: 0.000160589	valid_1's l2: 0.00105469
[4000]	training's l2: 0.000106139	valid_1's l2: 0.0010466
Early stopping, best iteration is:
[4496]	training's l2: 8.69548e-05	valid_1's l2: 0.00104382


<h2>预测</h2>

In [67]:
for site, df in df_test.groupby("光伏用户编号"):
    df = df.drop(columns=["光伏用户编号"]).astype(np.float32)
    y = df.pop("target")
    df["辐照强度（J/m2） - 1"] = df["辐照强度（J/m2）"].shift(1)
    
    y_pred = np.zeros((df.shape[0], ))
    for model in model_lgb[site]:
        y_pred += model.predict(df, num_iteration=model.best_iteration)
    y_pred = y_pred / kfold.n_splits
    df_test.loc[df_test["光伏用户编号"] == site, "target"] = y_pred

In [68]:
df_test = df_test[["光伏用户编号", "综合倍率", "年", "月", "日", "分", "target"]]
df_test["时间"] = df_test["年"].astype(str) + "-" + df_test["月"].astype(str) + "-" + df_test["日"].astype(str) + " 0:00"
df_test["分"] = "p" + (df_test["分"] + 1).astype(str)
df_test = df_test.drop(columns=["年", "月", "日"])

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [69]:
result = pd.pivot(df_test, index=["光伏用户编号", "综合倍率", "时间"], columns="分", values="target").reset_index()
result = result[result["综合倍率"].notnull()]
result["综合倍率"] = result["综合倍率"].astype(int)

In [70]:
result.to_csv("../data/%s.csv" % datetime.now().strftime("%Y%m%d_%H%M%S"), encoding="utf-8", index=False)